# Financial Sentiment Analysis
In this notebook I did Financial Sentiment Analysis by using different models
- First, Classification Models: Naive Bayes Classification Model
- Then, Sequential Models: RNN, LSTM, GRU
- Finally, Transformers: Bert

By the end of the notebook there's a comparison between results of each model

In [ ]:
# Essentials
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

## 1) Reading Data

In [ ]:
data = pd.read_csv("../input/financial-sentiment-analysis/data.csv")
data.head()

In [ ]:
data['Sentiment'].value_counts().plot(kind='bar',figsize=(10,6),grid='-')
plt.show()

In [ ]:
data['Sentiment'].value_counts()

> It seems that data is not balanced

## 2) Preprocessing

### 2.1) Data Cleaning

In [ ]:
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer() 
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

In [ ]:
def text_processing(text):   
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)
    
    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)
    
    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)
    
    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # remove special characters from text
    text = re.sub(r'\W', ' ', text)
    
    # tokenize the words using nltk word tokenizer and remove the stop words using nltk package's english stop words
    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    
    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])
    
    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [ ]:
data_copy = data.copy()
data_copy.Sentence = data_copy.Sentence.apply(lambda text: text_processing(text))

In [ ]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [ ]:
processed_data = {
    'Sentence':data_copy.Sentence,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

In [ ]:
processed_data['Sentiment'].value_counts()

### 2.2) Data Resampling
Since, Data is not balanced we need to do resampling to get balanced data

In [ ]:
df_neutral = processed_data[(processed_data['Sentiment']==1)] 
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative, 
                                 replace=True,    
                                 n_samples= 3130, 
                                 random_state=42)  

df_positive_upsampled = resample(df_positive, 
                                 replace=True,    
                                 n_samples= 3130, 
                                 random_state=42)  


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral,df_positive_upsampled])

In [ ]:
final_data['Sentiment'].value_counts()

## 3. Classification with Classifier Model (Naive Bayes)

### 3.1) Data Transformation

In [ ]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=1500)

In [ ]:
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

### 3.2) Dividing data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### 3.3) Model fitting

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
classifier = GaussianNB()
classifier.fit(X_train, y_train)

### 3.4) Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
nb_score = accuracy_score(y_test, y_pred)
nb_score

## 4) RNN, LSTM, GRU

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 4.1) Data Preparation

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(final_data['Sentence'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [ ]:
X_final = tokenizer.texts_to_sequences(final_data['Sentence'].values)
X_final = pad_sequences(X_final, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_final.shape)

In [ ]:
Y_final = pd.get_dummies(final_data['Sentiment']).values
print('Shape of label tensor:', Y_final.shape)

### 4.2) Dividing data into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size = 0.3, random_state = 42, shuffle=True)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

### 4.3) Model Training

#### 4.3.1) RNN

In [ ]:
rnn_model = Sequential()
rnn_model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_final.shape[1]))
rnn_model.add(SpatialDropout1D(0.2))
rnn_model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
rnn_model.add(Dense(3, activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

rnn_history = rnn_model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

#### 4.3.1) LSTM

In [ ]:
lstm_model = Sequential()
lstm_model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_final.shape[1]))
lstm_model.add(SpatialDropout1D(0.2))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

lstm_history = lstm_model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

#### 4.3.1) GRU

In [ ]:
gru_model = Sequential()
gru_model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_final.shape[1]))
gru_model.add(SpatialDropout1D(0.2))
gru_model.add(GRU(128, dropout=0.2, recurrent_dropout=0.2))
gru_model.add(Dense(3, activation='softmax'))
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

gru_history = gru_model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

### 4.4) Model Evaluation

In [ ]:
rnn_accuracy = rnn_model.evaluate(X_test,Y_test)
print('RNN Model Evaluation\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(rnn_accuracy[0],rnn_accuracy[1]))

In [ ]:
lstm_accuracy = lstm_model.evaluate(X_test,Y_test)
print('LSTM Model Evaluation\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(lstm_accuracy[0],lstm_accuracy[1]))

In [ ]:
gru_accuracy = gru_model.evaluate(X_test,Y_test)
print('GRU Model Evaluation\n Loss: {:0.3f}\n Accuracy: {:0.3f}'.format(gru_accuracy[0],gru_accuracy[1]))

#### 4.5) Plotting Results

In [ ]:
# def get_plot(trained_model):
#     accuracy=trained_model.history['accuracy']
#     loss=trained_model.history['loss']
#     val_accuracy=trained_model.history['val_accuracy']
#     val_loss=trained_model.history['val_loss']
#     epochs=[i for i in range(1,6)]

#     plt.figure(figsize=(12,8))
#     plt.plot(epochs,accuracy,'r',label='Training')
#     plt.plot(epochs,val_accuracy,'--',label='Validation')
#     plt.legend()
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.title('Accuracy',fontsize=15)
#     plt.grid()
#     plt.show()

#     plt.figure(figsize=(12,8))
#     plt.plot(epochs,loss,'r',label='Training')
#     plt.plot(epochs,val_loss,'--',label='Validation')
#     plt.legend()
#     plt.title('Loss',fontsize=15)
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.grid()
#     plt.show()

## BERT

In [ ]:
from transformers import AutoTokenizer, TFBertModel

bert_tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
bert_model = TFBertModel.from_pretrained("bert-large-uncased")

In [ ]:
print("Max length of Sentences:", max([len(x.split()) for x in final_data['Sentence']]))
print("Min length of Sentences:", min([len(x.split()) for x in final_data['Sentence']]))

In [ ]:
from sklearn.model_selection import train_test_split
X_train_bert, X_test_bert, Y_train_bert, Y_test_bert = train_test_split(final_data['Sentence'], final_data['Sentiment'], test_size = 0.25, random_state = 0, shuffle=True)
print(X_train_bert.shape,Y_train_bert.shape)
print(X_test_bert.shape,Y_test_bert.shape)

In [ ]:
X_bert_train_final = bert_tokenizer(
    text=X_train_bert.tolist(),
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_bert_train_final['input_ids'].shape

In [ ]:
X_bert_train_final.keys()

In [ ]:
Y_bert_test_final = pd.get_dummies(Y_test_bert).values
Y_bert_train_final = pd.get_dummies(Y_train_bert).values
Y_bert_train_final.shape, Y_bert_test_final.shape

In [ ]:
Y_train_bert.value_counts()

### 5.2) Build Model

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, BinaryAccuracy
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import tensorflow_hub as hub

In [ ]:
MAX_LENGTH = 50
INPUT_IDs = Input(shape=(MAX_LENGTH,), dtype=tf.int32, name="input_ids")
INPUT_MASK = Input(shape=(MAX_LENGTH,), dtype=tf.int32, name="attention_mask")

In [ ]:
embeddings = bert_model([INPUT_IDs, INPUT_MASK])[1] # 0 is hidden state, 1 is pooler_output
# shape of hidden state is (None, MAX_LENGTH, 1024) ==> out = tf.keras.layers.GlobalMaxPool1D(0.1)(embeddings)
# shape of booled output is (None, 1024) ==> out = tf.keras.layers.Dropout(0.1)(embeddings)

out = tf.keras.layers.Dropout(0.2)(embeddings)
out = Dense(64, activation="relu")(out)
out = tf.keras.layers.Dropout(0.2)(out)
out = Dense(32, activation="relu")(out)

y = Dense(3, activation='softmax')(out)
model = tf.keras.Model(inputs=[INPUT_IDs, INPUT_MASK], outputs=y)
model.layers[2].trainable = True

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# fit model
bert_history = model.fit(
    x={'input_ids': X_bert_train_final["input_ids"], 'attention_mask': X_bert_train_final["attention_mask"]},
    y=Y_bert_train_final,
    validation_split=0.15,
    epochs=5,
    batch_size=24
)

In [ ]:
X_test_bert_final = bert_tokenizer(
    text=X_test_bert.tolist(),
    add_special_tokens=True,
    max_length=50,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [ ]:
prediction = model.predict(
    x={'input_ids':X_test_bert_final["input_ids"], 'attention_mask':X_test_bert_final["attention_mask"]}
)

In [ ]:
prediction.shape

In [ ]:
y_pred = []
for i in range(len(prediction)):
    y_pred.append(tf.one_hot(tf.argmax(prediction[i]), 3)) 
y_pred = np.array(y_pred)

In [ ]:
y_pred[:3]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

accuracy_score(Y_bert_test_final, y_pred)

## Conclusion

In [ ]:
nb_acc = nb_score * 100
rnn_acc = rnn_accuracy[1] * 100
lstm_acc = lstm_accuracy[1] * 100
gru_acc = gru_accuracy[1] * 100

In [ ]:
import seaborn as sns
methods = ["Naive Bayes","RNN", "LSTM", "GRU"]
accuracy = [nb_acc, rnn_acc, lstm_acc, gru_acc]

sns.set()
plt.figure(figsize=(12,6))
plt.title("Models Summary")
plt.ylabel("Testing Accuracy %")
plt.xlabel("Model")
sns.barplot(x=methods, y=accuracy, palette="deep")

for idx, method in enumerate(methods):
    plt.text(idx -0.1, accuracy[idx]+0.02, "{:.2f}%".format(accuracy[idx]))
    
plt.tight_layout()

> As shown, GRU and LSTM got the highest accuracy score